In [13]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

import os, sys

#from arrow import get
sys.path.append('/home/hrai/codes/hpe_library')
from lib_import import *
from my_utils import BatchCamera, Camera
from my_utils import arg_as_list, get_configs
from my_utils import generate_random_segment, generate_random_trajectory, get_aligned_init_torso, get_ap_pose_2d, get_backbone_line_from_torso, get_bounded_segments_idx, get_cam_param, get_input, get_label, get_model_input, get_output, get_pairs, get_part_traj, get_pose_seq_and_cam_param, get_two_point_parts, load_h36m, load_segment_file_from_parameters, make_input, make_one_dimension_list, make_output, split_continuous_indices, MyCustomDataset, parse_args_by_model_name, get_input_gt_for_onevec, get_limb_angle, get_h36m_camera_info, h36m_data_processing
from my_utils import DH_matrix, azim_elev_to_vec, batch_azim_elev_to_vec, batch_inverse_tf, batch_projection, batch_rot_x_matrix, batch_rot_y_matrix, batch_rot_z_matrix, calculate_azimuth_elevation, calculate_rotation_quaternion, draw_arm, draw_subline, draw_torso_direction, frame_vec_to_matrix, generate_batch_tf_from_batch_origin_R, generate_camera_frame, generate_dh_frame, generate_tf_from_origin_R, generate_two_link, generate_world_frame, get_batch_frame_vec_from_keypoints, get_batch_lower_torso_frame_from_keypoints, get_batch_lower_torso_frame_from_pose, get_batch_reference_frame, get_batch_upper_torso_frame_from_keypoints, get_batch_upper_torso_frame_from_pose, get_frame_from_keypoints, get_lower_torso_frame_from_pose, get_reference_frame, get_torso_direction, get_torso_rotation_matrix, get_torso_shape, get_upper_torso_frame_from_pose, inverse_tf, normalize_vector, project_batch_tensor, projection, rotate_torso_by_R, rotate_torso_by_R_for_batch_tensor, rotation_matrix_to_vector_align, rotation_matrix_torso2torso, Appendage, BatchAppendage, BatchDHModel, DHModel, calculate_batch_azimuth_elevation
from my_utils import args_dict_to_namespace, construct_torso_from_output, denormalize_motionbert_result, get_dataset_info_from_segment_folder, get_inference_from_dhdst, get_inference_from_dhdst_torso, get_inference_from_motionbert, get_output_type, get_result, infer_one_segment, load_best_model_for_inference, normalize_input, test_model_by_segment_file
from my_utils import get_logger, log_configs
from my_utils import init_weights, split_array_by_idxs, BaselineModel, Linear, TorsoModel
from my_utils import Camera2ImageCoordinate, MPJPE, MPJPE_for_multiple_pose, MPJPE_for_single_pose, World2CameraCoordinate, World2ImageCoordinate, _sqrt_positive_part, _weak_project, aihub2h36m, array2dict, avgErrorForOneAction, avgErrorForOneActor, avgErrorForOneCamera, camera_to_image_frame, change_bbox_convention, check_max_min, coco2h36m, convert3DResult, draw_skeleton, draw_skeleton_2d, draw_skeleton_both, euclidean_distance, fit3d2h36m, fit3d_load_gt_and_param, get3DResult, getAIHubCameraParameter, getGT, getNumFromImgName, get_batch_h36m_keypoints, get_bbox_area, get_bbox_area_from_pose2d, get_bbox_from_pose2d, get_h36m_keypoint_index, get_h36m_keypoints, get_rootrel_pose, get_video_frame, get_video_info, get_xy_centered_pose, halpe2h36m, infer_box, kookmin2h36m, kookmin2h36m_with_nose, loadAIHubCameraParameter, matrix_to_quaternion, normalize, normalize_array, optimize_scaling_factor, plot_cv2_image, readJSON, readpkl, savepkl, skew_symmetric_matrix, skew_symmetric_matrix_tensor, standardize_quaternion, mpi_inf_3dhp2h36m, get_pose_height, get_batch_bbox_from_pose2d, len_ids, part_ids, h36m_part_keypoints, get_h36m_limb_lens, procrustes_align
from my_utils import check_duplicate_training, get_input_output_candidate, get_num_trial, load_args, load_best_model, load_dataset, load_model, prepare_training, run, run_epoch, save_args, split_array_by_idxs
from my_utils import axes_2d, axes_3d, axes_to_compare_pred_gt, draw_3d_pose, draw_bbox, draw_multiple_2d_pose, draw_multiple_3d_pose, draw_one_segment, draw_rotation_matrix, draw_segment, draw_segments, draw_trajectory, get_2d_pose_image, legend_without_duplicate_labels, plot_to_compare_pred_gt, save_h36m_pose_video, show2Dtrajectory, show3Dtrajectory, show_2d_3d, show_whole_segment_trajectories, draw_2d_pose, clear_axes
from my_utils import check_available_frame, check_continuity, draw_base_marker_3d, generate_kookmin_pkl_for_each_video, get_cam_param_kookmin, get_video_frame_kookmin, get_video_num_frame_kookmin, load_pose3d_kookmin, load_csv_kookmin, get_lbot
os.chdir('/home/hrai/codes/MotionBERT')

from lib.utils.tools import * # get_config
from lib.utils.learning import * # load_backbone
from lib.utils.args import parse_args, check_args
from lib.utils.utils_data import flip_data
from lib.model.loss import *
from lib.model.evaluation import *
from lib.data.datareader_kookmin import DataReaderKOOKMIN
from lib.data.datareader_h36m import DataReaderH36M
from lib.data.datareader_random_limb import DataReaderRandomLimb
from lib.data.dataset_motion_3d import MotionDataset3D
from lib.model.DHDSTformer import DHDSTformer_total, DHDSTformer_total2, DHDSTformer_total3, DHDSTformer_limb, DHDSTformer_torso, DHDSTformer_torso2, DHDSTformer_total4

import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display
from ipywidgets import interact, interactive, fixed, interact_manual, interactive_output
import threading
from ipywidgets import GridspecLayout
from ipywidgets import TwoByTwoLayout

from utils_result_analysis.button import get_inference_button, get_visualize_button, get_root_rel_button, get_reset_button, get_go_to_max_frame_button, get_procrustes_button, get_analysis_error_button, get_analysis_dh_button
from utils_result_analysis.progress import get_inference_progress
from utils_result_analysis.text import get_inputs_all_shape_text, get_gts_all_shape_text, get_results_all_shape_text, get_error_max_frame_text, get_batch_num_text, get_frame_num_text
from utils_result_analysis.slider import get_azim_slider, get_elev_slider, get_zoom_slider, get_delay_slider, get_frame_slider
from utils_result_analysis.dropdown import get_model_list_dropdown, get_dataset_list_dropdown
from utils_result_analysis.play import get_play_vis_button
from utils_result_analysis.select import get_subject_select, get_action_select, get_cam_select, get_batch_select, get_part_select, get_part_select2

box_layout = widgets.Layout(
        border='solid 1px red',
        margin='0px 10px 10px 0px',
        padding='5px 5px 5px 5px')
plt.clf()
plt.cla()

class analysis_dataset:
    def __init__(self):
        self.right_arm_id = 0
        self.left_arm_id = 1
        self.right_leg_id = 2
        self.left_leg_id = 3
        self.test_out = widgets.Output()
        try:
            # model, data preparation -----------------------------------------------------
            self.load_panel()
            # select ---------------------------------------------------------------------
            self.select_panel()
            # figure ---------------------------------------------------------------------
            self.figure_panel()
            # interactive -----------------------------------------------------------------
            self.interactive()
            # layout ----------------------------------------------------------------------
            self.layout()
            # main
            self.main()
        except Exception as e:
            #with self.test_out:
            print(e)

    # panel functions -------------------------------------------------------------------  
    def load_panel(self):
        self.dataset_list = get_dataset_list_dropdown(['H36M'])

    def select_panel(self):
        self.select_subject = get_subject_select([''])
        self.select_action = get_action_select([''])
        #self.select_cam = get_cam_select([''])
        #self.select_batch = get_batch_select([''])
        self.select_subject.layout.width = 'max-content'
        self.select_subject.rows = 7
        self.select_action.layout.width = 'max-content'
        self.select_action.rows = 7
        #self.select_cam.layout.width = 'max-content'
        #self.select_batch.layout.width = 'max-content'
        self.select_part = get_part_select2(['right_arm', 'left_arm', 'right_leg', 'left_leg'])
        self.select_part.layout.width = 'max-content'

    def figure_panel(self):
        self.plot_3d = widgets.Output(layout=box_layout)
        with self.plot_3d:
            self.fig_3d = plt.figure(1, figsize=(10, 10), layout='constrained')
            self.fig_3d.clear()
            self.fig_3d.tight_layout()
            self.ax_3d1 = axes_3d(self.fig_3d, [0, 0, 1, 1], xlim=(-2,2), ylim=(-2,2), zlim=(0,2), view=(10,-90), show_axis=False)      
            self.ax_3d1.set_aspect('equal')    
            f = plt.show()
        self.frame_slider = get_frame_slider(1)
        self.frame_slider.layout.width = 'auto'

        self.plot_angle = widgets.Output(layout=box_layout)
        with self.plot_angle:
            self.fig_angle = plt.figure(2, figsize=(15,5))
            self.fig_angle.clear()
            #self.fig_angle.tight_layout()
            self.ax_angle = self.fig_angle.add_subplot(111)  
            #self.ax_angle.set_aspect('equal')    
            f = plt.show()

    def load_data(self, target_dataset):
        self.target_dataset = target_dataset
        if target_dataset == 'H36M':
            self.world_3d, self.h36m_cam_param = load_h36m()
            self.subject_list = list(self.world_3d._data.keys())
            self.select_subject.options = self.subject_list
            if self.select_subject.value == '':
                self.select_subject.value = self.select_subject.options[0]
            self.select_action.options = list(self.world_3d._data[self.select_subject.value].keys())
            if self.select_action.value == '':
                self.select_action.value = self.select_action.options[0]
            self.frame_slider.max = self.world_3d._data[self.select_subject.value][self.select_action.value]['positions'].shape[0]
            print(f'{target_dataset} Loaded!')
        else:
            print(f'{target_dataset} is not implemented yet')

    def update_select(self, subject):
        try:
            self.select_action.options = list(self.world_3d._data[subject].keys())
            self.select_action.value = self.select_action.options[0]
        except Exception as e:
            with self.test_out:
                print('select error', e)

    def update_pose_3d(self, action):
        # update pose 3d list
        self.pose_3d_list = self.world_3d._data[self.select_subject.value][action]['positions'].copy()
        # generate dh model
        self.dh_model = DHModel(self.pose_3d_list[0].copy(), forward_dir='-z')
        # update frame slider
        self.frame_slider.max = self.pose_3d_list.shape[0]
        # get dh angles of all frame
        self.dh_angles = []
        for i in range(0, len(self.pose_3d_list)):
            pose_3d = self.pose_3d_list[i]
            self.dh_model.set_dh_model_from_pose(pose_3d)
            self.dh_angles.append(self.dh_model.get_dh_angles(by_dict=False))
        self.dh_angles = np.array(self.dh_angles)
        self.dh_angles_diff = self.dh_angles.copy()[:-1] - self.dh_angles.copy()[1:]
        self.visualize_angle(self.select_part.value)
        self.update_frame_data(self.frame_slider.value)
        with self.test_out:
            print(f"{self.select_subject.value} {action} pose 3d updated")

    def update_frame_data(self, frame): # self.frame_num.value
        try: 
            with self.test_out:
                print(frame, self.frame_slider.value, end='\r')
            frame_num = frame-1
            #self.frame_input_2d = self.batch_input[frame_num].copy()
            self.pose_3d = self.pose_3d_list[frame_num].copy()
            self.dh_model.set_dh_model_from_pose(self.pose_3d)
            # updtate figure
            self.visualize_3d(frame_num)
        except Exception as e:
            with self.test_out:
                print(e)

    def visualize_3d(self, frame):
        try: 
            with self.plot_3d:
                plt.sca(self.ax_3d1)
                clear_axes(self.ax_3d1)
                #self.ax_3d1.set_title(f'frame {frame} GT - action: {self.select_action.value}')
                center_point = self.pose_3d[0].copy()
                #self.ax_3d1.set_xlim(center_point[0]-0.5, center_point[0]+0.5)
                #self.ax_3d1.set_ylim(center_point[1]-0.5, center_point[1]+0.5)
                #self.ax_3d1.set_zlim(center_point[2]-1, center_point[2]+1)
                self.ax_3d1.set_aspect('equal')
                draw_3d_pose(self.ax_3d1, self.pose_3d, alpha=0.5)
                self.dh_model.draw(self.ax_3d1, draw_frame=True, show_axis=False, scale=0.1, fontsize=5)
        except Exception as e:
            with self.test_out:
                print(e)

    def visualize_angle(self, part):
        with self.test_out:
            print(f'visualize_angle {part}')
        try:
            if part == 'right_arm': part_id = self.right_arm_id
            elif part == 'left_arm': part_id = self.left_arm_id
            elif part == 'right_leg': part_id = self.right_leg_id
            elif part == 'left_leg': part_id = self.left_leg_id
            link1_azim = self.dh_angles[:, part_id*4+0]
            link1_elev = self.dh_angles[:, part_id*4+1]
            link2_azim = self.dh_angles[:, part_id*4+2]
            link2_elev = self.dh_angles[:, part_id*4+3]
            link1_azim_diff = self.dh_angles_diff[:, part_id*4+0]
            link1_elev_diff = self.dh_angles_diff[:, part_id*4+1]
            link2_azim_diff = self.dh_angles_diff[:, part_id*4+2]
            link2_elev_diff = self.dh_angles_diff[:, part_id*4+3]
            with self.test_out:
                print(link1_azim.shape, link1_elev.shape, link2_azim.shape, link2_elev.shape)
            with self.plot_angle:
                plt.sca(self.ax_angle)
                #clear_axes(self.ax_angle)
                self.ax_angle.clear()
                # self.ax_angle.plot(np.arange(len(link1_azim)), link1_azim)
                # self.ax_angle.plot(np.arange(len(link1_elev)), link1_elev)
                # self.ax_angle.plot(np.arange(len(link2_azim)), link2_azim)
                # self.ax_angle.plot(np.arange(len(link2_elev)), link2_elev)
                self.ax_angle.plot(np.arange(len(link1_azim_diff)), link1_azim_diff, label='link1_azim_diff')
                self.ax_angle.plot(np.arange(len(link1_elev_diff)), link1_elev_diff, label='link1_elev_diff')
                self.ax_angle.plot(np.arange(len(link2_azim_diff)), link2_azim_diff, label='link2_azim_diff')
                self.ax_angle.plot(np.arange(len(link2_elev_diff)), link2_elev_diff, label='link12elev_diff')

                self.ax_angle.set_xticks(np.arange(0, len(link1_azim), 50))
                self.ax_angle.grid()
                self.ax_angle.legend()
                
        except Exception as e:
            with self.test_out:
                print(e)

    def interactive(self): 
        self.load_dataset_interact = widgets.interactive_output(self.load_data, {'target_dataset': self.dataset_list})
        self.select_interact = widgets.interactive(self.update_select, subject=self.select_subject)
        self.update_pose_interact = widgets.interactive_output(self.update_pose_3d, {'action': self.select_action})
        self.frame_interact = widgets.interactive(self.update_frame_data, frame=self.frame_slider)
        self.part_interact = widgets.interactive(self.visualize_angle, part=self.select_part)

    def layout(self):
        ui_prepare = widgets.VBox([self.dataset_list, self.load_dataset_interact], layout=box_layout, width='auto')
        ui_select = widgets.VBox([self.select_subject, self.select_action, self.select_part], layout=box_layout)
        ui_figure = widgets.HBox([self.plot_angle, ui_select], layout=box_layout, width='auto')
        ui_level1 = widgets.HBox([ui_prepare, widgets.VBox([self.plot_3d, self.frame_slider])])
        self.ui = widgets.VBox([ui_level1, ui_figure, self.test_out])

    def main(self, b=0):
        dis = display(self.ui)

ad = analysis_dataset()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
a = np.array([1, 2, 3, 4, 5])
b = a.copy()[1:]
# b = np.roll(b, -1)
# b[-1] = b[-2]
a[:-1] - b

array([-1, -1, -1, -1])